In [2]:
import pandas as pd

In [3]:
file_path = '../UP-1.0/en_ewt-up-dev.conllu'

In [4]:
with open(file_path, 'r') as f:
    lines = f.readlines()

In [5]:
len(lines)

31502

In [6]:
lines = [l for l in lines if not l.startswith('#') and len(l)>1]
len(lines)

25150

In [7]:
with open('../UP-1.0/dev.temp', 'w') as f:
    f.writelines(lines)

In [8]:
df_dev = pd.read_csv('../UP-1.0/dev.temp', sep='\t', on_bad_lines='skip')
df_dev.head()

,1,From,from,ADP,IN,_,3,case,3:case,_.1,_.2,_.3
0,2.0,the,the,DET,DT,Definite=Def|PronType=Art,3,det,3:det,_,_,_
1,3.0,AP,AP,PROPN,NNP,Number=Sing,4,obl,4:obl:from,_,_,ARG2
2,4.0,comes,come,VERB,VBZ,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,0,root,0:root,_,come.03,V
3,5.0,this,this,DET,DT,Number=Sing|PronType=Dem,6,det,6:det,_,_,_
4,6.0,story,story,NOUN,NN,Number=Sing,4,nsubj,4:nsubj,_,_,ARG1


In [13]:
voices = []
for i, row in df_dev.iterrows():
    if row[7] in ['nsubj:pass', 'aux:pass']:
        voices.append('passive')
    elif row[7] == 'nsubj':
        voices.append('active')
    else:
        voices.append('None')
df_dev['voice'] = voices

In [15]:
df_dev.head(10)

,1,From,from,ADP,IN,_,3,case,3:case,_.1,_.2,_.3,voice
0,2.0,the,the,DET,DT,Definite=Def|PronType=Art,3,det,3:det,_,_,_,None
1,3.0,AP,AP,PROPN,NNP,Number=Sing,4,obl,4:obl:from,_,_,ARG2,None
2,4.0,comes,come,VERB,VBZ,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,0,root,0:root,_,come.03,V,None
3,5.0,this,this,DET,DT,Number=Sing|PronType=Dem,6,det,6:det,_,_,_,None
4,6.0,story,story,NOUN,NN,Number=Sing,4,nsubj,4:nsubj,_,_,ARG1,active
5,7.0,:,:,PUNCT,:,_,4,punct,4:punct,_,_,_,None
6,1.0,***,***,PUNCT,NFP,_,0,root,0:root,_,_,NaN,None
7,1.0,A,a,X,FW,Foreign=Yes,0,root,0:root,_,_,NaN,None
8,2.0,la,la,X,FW,Foreign=Yes,1,flat:foreign,1:flat:foreign,_,_,NaN,None
9,3.0,guerre,guerre,X,FW,Foreign=Yes,1,flat:foreign,1:flat:foreign,_,_,NaN,None
